In [147]:
import pandas as pd
from datetime import datetime

In [148]:
import yahoo_fin.options as ops

In [158]:
aapl_div = 0.22
sofr = 0.27
ex_div_date = datetime(2022,5,6)
pay_date = datetime(2022,5,12)
today = datetime.now()
expiration = datetime(2022,5,20)
type = "call"
strike_price = [100,110,120,130,140,150]

In [159]:
if type == "call":
    output = pd.DataFrame(columns = ['Contract Name','Type','Strike' , 'Early Exercise'])
    df = ops.get_puts("aapl", date = expiration)
    df["Exercise Price"] = df["Strike"]
    option_detail = df.drop("Implied Volatility",axis = 1).set_index("Strike")
    for i in strike_price:
        aapl_div_with_int = aapl_div*(1+sofr*((expiration - pay_date).days/360))
        exercise = aapl_div_with_int > option_detail.at[i,"Ask"] + i*sofr/100*((expiration - today).days/360)
        temp = {"Contract Name":option_detail.at[i,"Contract Name"],"Type":"Call", "Strike":i,"Early Exercise": exercise}
        output = output.append(temp, ignore_index= True)
else:
    pass

In [160]:
if type == "put":
    output = pd.DataFrame(columns = ['Contract Name','Type','Strike' , 'Early Exercise', "Blackout Period"])
    df2 = ops.get_calls("aapl", date = expiration)
    df2["Exercise Price"] = df2["Strike"]
    option_detail = df2.drop("Implied Volatility",axis = 1).set_index("Strike")
    for i in strike_price:
        aapl_div_with_int = aapl_div*(1+sofr*((expiration - pay_date).days/360))
        exercise = i*sofr/100*((expiration - today).days/360) > aapl_div_with_int + option_detail.at[i,"Ask"]
        black_out = aapl_div / (i*sofr/100*(1/360))
        temp = {"Contract Name":option_detail.at[i,"Contract Name"],"Type":"Put", "Strike":i,"Early Exercise": exercise, "Blackout Period": black_out}
        output = output.append(temp, ignore_index= True)
else:
    pass

In [161]:
output

,Contract Name,Type,Strike,Early Exercise
0,AAPL220520P00100000,Call,100,True
1,AAPL220520P00110000,Call,110,False
2,AAPL220520P00120000,Call,120,False
3,AAPL220520P00130000,Call,130,False
4,AAPL220520P00140000,Call,140,False
5,AAPL220520P00150000,Call,150,False


In [153]:
# output.to_excel("EarlyExerciseCandidate.xlsx")